In [125]:
from langchain.utilities.duckduckgo_search import DuckDuckGoSearchAPIWrapper
import yfinance
import json


def get_ticker(inputs):
    ddg = DuckDuckGoSearchAPIWrapper()
    company_name = inputs["company_name"]
    return ddg.run(f"Ticker symbol of {company_name}")


def get_income_statement(inputs):
    ticker = inputs["ticker"]
    stock = yfinance.Ticker(ticker)
    return json.dumps(stock.income_stmt.to_json())


def get_balance_sheet(inputs):
    ticker = inputs["ticker"]
    stock = yfinance.Ticker(ticker)
    return json.dumps(stock.balance_sheet.to_json())


def get_daily_stock_performance(inputs):
    ticker = inputs["ticker"]
    stock = yfinance.Ticker(ticker)
    return json.dumps(stock.history(period="3mo").to_json())


functions_map = {
    "get_ticker": get_ticker,
    "get_income_statement": get_income_statement,
    "get_balance_sheet": get_balance_sheet,
    "get_daily_stock_performance": get_daily_stock_performance,
}


functions = [
    {
        "type": "function",
        "function": {
            "name": "get_ticker",
            "description": "Given the name of a company returns its ticker symbol",
            "parameters": {
                "type": "object",
                "properties": {
                    "company_name": {
                        "type": "string",
                        "description": "The name of the company",
                    }
                },
                "required": ["company_name"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_income_statement",
            "description": "Given a ticker symbol (i.e AAPL) returns the company's income statement.",
            "parameters": {
                "type": "object",
                "properties": {
                    "ticker": {
                        "type": "string",
                        "description": "Ticker symbol of the company",
                    },
                },
                "required": ["ticker"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_balance_sheet",
            "description": "Given a ticker symbol (i.e AAPL) returns the company's balance sheet.",
            "parameters": {
                "type": "object",
                "properties": {
                    "ticker": {
                        "type": "string",
                        "description": "Ticker symbol of the company",
                    },
                },
                "required": ["ticker"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_daily_stock_performance",
            "description": "Given a ticker symbol (i.e AAPL) returns the performance of the stock for the last 100 days.",
            "parameters": {
                "type": "object",
                "properties": {
                    "ticker": {
                        "type": "string",
                        "description": "Ticker symbol of the company",
                    },
                },
                "required": ["ticker"],
            },
        },
    },
]

In [126]:
import openai as client

""" assistant = client.beta.assistants.create(
    name="Investor Assistant",
    instructions="You help users do research on publicly traded companies and you help users decide if they should buy the stock or not.",
    model="gpt-4-1106-preview",
    tools=functions,
) """
assistant_id = "asst_6v0MLruDKrutBQWbXl6RmVx0"

In [127]:
# openai assistants thread에 결과를 보내기 위해 사용.
# get_ticker({"company_name": "Tesla Inc"})

In [128]:
# get_income_statement({"ticker": "TSLA"})

In [129]:
# get_balance_sheet({"ticker": "TSLA"})

In [130]:
# get_daily_stock_performance({"ticker": "TSLA"})

In [157]:
thread = client.beta.threads.create(
    messages=[
        {"role": "user", "content": "Salesforce 를 사는게 좋은 선택일까?"}
    ]
)
thread

Thread(id='thread_lHYdafjjXhHSB10qzxCuwB9r', created_at=1735423105, metadata={}, object='thread', tool_resources=ToolResources(code_interpreter=None, file_search=None))

In [184]:
# 새로운 run을 만드는 것 ( 새롭게 응답을 시작하는 것! 새롭게 응답 하기에 openai가 이런 물음에는 이런 함수를 쓰셔야 합니다 하면서 알려줌! 🪄 )
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant_id,
)

run

BadRequestError: Error code: 400 - {'error': {'message': 'Thread thread_lHYdafjjXhHSB10qzxCuwB9r already has an active run run_R99hGwu4Y3O6wve056pxm7if.', 'type': 'invalid_request_error', 'param': None, 'code': None}}

In [133]:
def get_run(run_id, thread_id):
    return client.beta.threads.runs.retrieve(
        run_id=run_id,
        thread_id=thread_id,
    )


def send_message(thread_id, content):
    return client.beta.threads.messages.create(
        thread_id=thread_id, role="user", content=content
    )


def get_messages(thread_id):
    messages = client.beta.threads.messages.list(thread_id=thread_id)
    messages = list(messages)
    messages.reverse()
    for message in messages:
        print(f"{message.role}: {message.content[0].text.value}")


# assistant 가 원하는 output을 submit하는 함수. (openai assistants는 함수의 결과예시도 알려줘야함!)
def get_tool_outputs(run_id, thread_id):
    run = get_run(run_id, thread_id)
    outputs = []
    for action in run.required_action.submit_tool_outputs.tool_calls:
        action_id = action.id
        function = action.function
        # json.loads를 사용하는 이유는 응답이 text로 오기에, python에서 사용할 수 있도록 변환 해주는 과정이 필요하기 때문임.
        output = functions_map[function.name](json.loads(function.arguments))
        outputs.append({
            "output": output,
            "tool_call_id": action_id,
        })
    # 결과를 보내기 위해 call id(call_j6Yk2GlAEcoZCfsxTTqn1ARm)들의 결과를 return!
    return outputs


def submit_tool_outputs(run_id, thread_id):
    outputs = get_tool_outputs(run_id, thread_id)
    return client.beta.threads.runs.submit_tool_outputs(
        run_id=run_id,
        thread_id=thread_id,
        tool_outputs=outputs
    )

In [185]:
# get_run(run.id, thread.id).status
# requires_action 면 required action이 충족이 안되어 있음! 이라는것!
# queued 가 뜬다면... 그냥 느려서 그런거니 requires_action가 될떄까지 기다리면됨!.
get_run(run.id, thread.id).status

'requires_action'

In [191]:
# assistant가 원하는 함수를 알 수 있음!
get_tool_outputs(run.id, thread.id)

AttributeError: 'NoneType' object has no attribute 'submit_tool_outputs'

In [192]:
# assistant 가 원하는 부분을 보내줘야함!
submit_tool_outputs(run.id, thread.id)
# 'NoneType' object has no attribute 'submit_tool_outputs' 가 뜨는 이유는 openai가 답변을 기다리다가 응답 받는것을 닫아버렸기 떄문임. get_run(run.id, thread.id).status 를 해보면 expired되었다 라고 뜰거임!

AttributeError: 'NoneType' object has no attribute 'submit_tool_outputs'

In [194]:
get_run(run.id, thread.id).status

'completed'

In [195]:
get_messages(thread.id)

user: Salesforce 를 사는게 좋은 선택일까?
assistant: Salesforce (CRM)에 대한 분석을 통해 주식 구매의 적합성을 평가하겠습니다.

### 1. **재무 성과**
- **총 수익**: Salesforce의 총 수익은 $34,857,000,000 (약 348억 5천 7백만 달러)입니다.
- **순이익**: 순이익은 $4,136,000,000 (약 41억 3천 6백만 달러)이며, 이는 주당 순이익 (EPS)이 $4.20이라는 것을 나타냅니다.
- **총 비용**: 총 비용은 $28,858,000,000 (약 288억 5천 8백만 달러)입니다.

### 2. **재무 상태**
- **자산**: 총 자산은 $99,823,000,000 (약 998억 2천 3백만 달러)입니다.
- **부채**: 총 부채는 $12,588,000,000 (약 125억 8천 8백만 달러)으로, 상대적으로 관리 가능한 수준입니다.
- **주주 자본**: 주주 자본은 $59,646,000,000 (약 596억 4천 6백만 달러)으로 기업의 재정적 안정성을 나타냅니다.

### 3. **주식 성과**
금일 기준, Salesforce의 주가는 최근 100일 동안 다음과 같은 성과를 보였습니다:
- **현재 주가**: 약 $344 (정확한 값은 확인 필요)
- **최근 저점**: $273.40
- **최근 고점**: $368.57

최근 이 주식은 상승세를 보였으며, 변동성도 있는 편입니다.

### 결론
Salesforce는 안정적인 순이익과 강력한 매출 성장을 보이고 있습니다. 그러나 최근 주가는 고공행진하고 있으며, 이로 인해 주식에 대한 평가가 높아질 수 있습니다. 당신이 이 주식을 구매하기에 적합한지는 귀하의 투자 목표와 위험 선호도에 따라 달라질 수 있습니다. 안정성보다 성장 가능성에 중점을 두신다면, Salesforce는 좋은 선택일 수 있습니다.

하지만 지금 당장 주가가 높은 점을 고려한다면, 시간을 두고 관찰하거나 추가적인 분석을 통해 구매 결정을 

In [183]:
send_message(thread.id, "이제 Cloudflare를 사는것은 좋은 선택인지 알고 싶습니다.")

BadRequestError: Error code: 400 - {'error': {'message': "Can't add messages to thread_lHYdafjjXhHSB10qzxCuwB9r while a run run_R99hGwu4Y3O6wve056pxm7if is active.", 'type': 'invalid_request_error', 'param': None, 'code': None}}

In [188]:
# 그냥 메세지가 추가된 것이라 아무일이 없음! 실제로 thread를 실행해야 뭔가 일어나는 거임!
get_messages(thread.id)

user: Salesforce 를 사는게 좋은 선택일까?
assistant: Salesforce (CRM)에 대한 분석을 통해 주식 구매의 적합성을 평가하겠습니다.

### 1. **재무 성과**
- **총 수익**: Salesforce의 총 수익은 $34,857,000,000 (약 348억 5천 7백만 달러)입니다.
- **순이익**: 순이익은 $4,136,000,000 (약 41억 3천 6백만 달러)이며, 이는 주당 순이익 (EPS)이 $4.20이라는 것을 나타냅니다.
- **총 비용**: 총 비용은 $28,858,000,000 (약 288억 5천 8백만 달러)입니다.

### 2. **재무 상태**
- **자산**: 총 자산은 $99,823,000,000 (약 998억 2천 3백만 달러)입니다.
- **부채**: 총 부채는 $12,588,000,000 (약 125억 8천 8백만 달러)으로, 상대적으로 관리 가능한 수준입니다.
- **주주 자본**: 주주 자본은 $59,646,000,000 (약 596억 4천 6백만 달러)으로 기업의 재정적 안정성을 나타냅니다.

### 3. **주식 성과**
금일 기준, Salesforce의 주가는 최근 100일 동안 다음과 같은 성과를 보였습니다:
- **현재 주가**: 약 $344 (정확한 값은 확인 필요)
- **최근 저점**: $273.40
- **최근 고점**: $368.57

최근 이 주식은 상승세를 보였으며, 변동성도 있는 편입니다.

### 결론
Salesforce는 안정적인 순이익과 강력한 매출 성장을 보이고 있습니다. 그러나 최근 주가는 고공행진하고 있으며, 이로 인해 주식에 대한 평가가 높아질 수 있습니다. 당신이 이 주식을 구매하기에 적합한지는 귀하의 투자 목표와 위험 선호도에 따라 달라질 수 있습니다. 안정성보다 성장 가능성에 중점을 두신다면, Salesforce는 좋은 선택일 수 있습니다.

하지만 지금 당장 주가가 높은 점을 고려한다면, 시간을 두고 관찰하거나 추가적인 분석을 통해 구매 결정을 

## 메모! (순서대로 정리 해보기)

1. 새로운 thread를 만든다 (새로운 주제)
2. 새로운 run을 만든다 (실행)
3. openai가 컴퓨터에서 돌릴 함수를 찾아주고 실행해서 응답을 돌라함!
4. 필요한 함수를 실행하고, 그 응답을 assistant에게 보냄.
5. assistant는 받은 응답을 가지고 정답을 생성하고. 다 끝난다면 complate상태가 됨!
6. 응답을 받음..
7. 만약 새로운 message가 온다면...
   - 1. 새로운 run을 만든다
   - 2. 새로운 run을 실행하고, 함수를 받고 실행하고 반복한다!

### 참조

- thread는 user와 assistant간의 대화임.(계속 살려두어야 계속 참조가 가능함. 코드를 다시 실행하면 사라짐! .. run도 마찬가지)
- run는 말그대로 실행임 이때 실행은 thread를 실행하는 것임. thread는 위에서 getmessage로 확인 가능 하듯이 message를 가지고 있음. 그리고 실행한 후에는 알려준 함수중에서 어떤 함수를 써야 한다를 가지고있고, 그걸 실행할 떄까지 기다려줌. 너무 오래 걸리면 어디론가 가버리지만..
- [wait_on_run](https://cookbook.openai.com/examples/assistants_api_overview_python#runs)

### Code Challenge

- streamlit 으로 구현해보기!
